#### 3.1. Representing a game of Go in Python
The game of Go is played on a square board. Usually, beginners start playing on a 9 × 9 or 13 × 13 board, and advanced and pro players play on a 19 × 19 board. But in principle, Go can be played on a board of any size. Implementing a square grid for the game is fairly simple, but you’ll need to take care of a lot of intricacies down the line.

You represent a Go game in Python by building a module we’ll call dlgo step-by-step. Throughout the chapter, you’ll be asked to create files and implement classes and functions that will eventually lead to your first bot. All the code from this and later chapters can be found on GitHub at http://mng.bz/gYPe.

Although you should definitely clone this repository for reference, we strongly encourage you to follow along by creating the files from scratch to see how the library builds up piece by piece. The master branch of our GitHub repository contains all the code used in this book (and more). From this chapter on, there’s additionally a specific Git branch for each chapter that has only the code you need for the given chapter. For instance, the code for this chapter can be found in branch chapter_3. The next chapters follow the same naming convention. Note that we’ve included extensive tests for most of the code found here and in later chapters in the GitHub repository.

To build a Python library to represent Go, you need a data model that’s flexible enough to support a few use cases:

- Track the progress of a game you’re playing against a human opponent.
- Track the progress of a game between two bots. This might seem to be exactly the same as the preceding point, but as it turns out, a few subtle differences exist. Most notably, naive bots have a hard time recognizing when the game is over. Playing two simple bots against each other is an important technique used in later chapters, so it’s worth calling out here.
- Compare many prospective sequences from the same board position.
- Import game records and generate training data from them.

First, create a new folder, dlgo, and place an empty \__init__.py file into it to initiate it as a Python module. Also, create two additional files called gotypes.py and goboard_slow.py in which you’ll put all board- and game-play functionality. Your folder structure at this point should look as follows:


__dlgo__\
$~~~~~~$ \_\___init____.$\color{brown}{\text{py}}$
\
$~~~~~~$ __gotypes__.$\color{brown}\text{py}$\
$~~~~~~$ __goboard_slow__.$\color{brown}\text{py}$

Black and white players take turns in Go, and you use `enum` to represent the different-colored stones. A `Player` is either `black` or `white`. After a player places a stone, you can switch the color by calling the `other` method on a `Player` instance. Put this `Player` class into gotypes.py.



In [288]:
# Listing 3.1. Using an enum to represent players

import enum


class Player(enum.Enum):
    """ 
    >>> Player.black.value
    1
    """
    black = 1
    white = 2

    @property
    def other(self):
        return Player.black if self == Player.white else Player.white

As noted in the front matter, we’re using Python 3 for this book. One of the reasons is that many modern aspects of the language, such as enums in gotypes.py, are part of the standard library in Python 3.

Next, to represent coordinates on the board, tuples are an obvious choice. The following `Point` class also goes into gotypes.py.

In [289]:
# Listing 3.2. Using tuples to represent points of a Go board
from collections import namedtuple


class Point(namedtuple('Point', 'row col')):
    """ 
    >>> p=Point(1,col=2)
    >>> p
    Point(row=1, col=2)
    >>> p.row
    1
    """
    def neighbors(self):
        return [
            Point(self.row - 1, self.col),
            Point(self.row + 1, self.col),
            Point(self.row, self.col - 1),
            Point(self.row, self.col + 1),
        ]

A `namedtuple` lets you access the coordinates as point.row and point.col instead of `point[0]` and `point[1]`, which makes for much better readability.

You also need a structure to represent the actions a player can take on a turn. Normally, a turn involves placing a stone on the board, but a player can also pass or resign at any time. Following American Go Association (AGA) conventions, we use the term move to mean any of those three actions, whereas a play refers to placing a stone. In the `Move` class, you therefore encode all three types of move (play, pass, resign) and make sure a move has precisely one of these types. For actual plays, you need to pass a `Point` to be placed. You add this `Move` class to the file goboard_slow.py.

In [290]:
# Listing 3.3. Setting moves: plays, passes, or resigns
import copy
from dlgo.gotypes import Player

class Move():
    def __init__(self, point=None, is_pass=False, is_resign=False):
        """ 
        It's a bitwise XOR (exclusive OR).
        It results to true if one (and only one) of the operands (evaluates to) true.
        >>> 0^0
        0
        >>> 1^1
        0
        >>> 1^0
        1
        >>> 0^1
        1
        >>> 0^1^0
        1
        >>> 0^1^1
        0
        >>> bin(8)
        '0b1000'
        >>> bin(3)
        '0b11'
        >>> bin(11)
        '0b1011'
        >>> 8^3
        11
         """
        assert (point is not None) ^ is_pass ^ is_resign
        self.point = point
        self.is_play = (self.point is not None)
        self.is_pass = is_pass
        self.is_resign = is_resign

    @classmethod
    def play(cls, point):
        return Move(point=point)

    @classmethod
    def pass_turn(cls):
        return Move(is_pass=True)

    @classmethod
    def resign(cls):
        return Move(is_resign=True)



In what follows, clients generally won’t call the `Move` constructor directly. Instead, you usually call `Move.play`, `Move.pass_turn`, or `Move.resign` to construct an instance of a move.

Note that so far, the `Player`, `Point`, and `Move` classes are all plain data types. Although they’re fundamental to representing the board, they don’t contain any game logic. This is done on purpose, and you’ll benefit from separating game-play concerns like this.

Next, you implement classes that can update the game state by using the preceding three classes:

- The `Board` class is responsible for the logic of placing and capturing stones.
- The `GameState` class includes all the stones of the board, as well as tracking whose turn it is and what the previous state was.

##### 3.1.1. Implementing the Go board

Before turning to `GameState`, let’s first implement the Board class. Your first idea might be to create a 19 × 19 array tracking the state of each point in the board, and that’s a good starting point. Now, think about the algorithm for checking when you need to remove stones from the board. Recall that the number of liberties of a single stone is defined by the number of empty points in its direct neighborhood. If all four neighboring points are occupied by an enemy stone, the stone has no liberties left and is captured. For larger groups of connected stones, the situation is more difficult to check. For instance, after placing a black stone, you have to check all neighboring white stones to see whether black captured any stones that you have to remove. Specifically, you have to check the following:

1. You see whether any neighbors have any liberties left.
2. You check whether any of the neighbors’ neighbors have any liberties left.
3. You examine the neighbors’ neighbors’ neighbors, and so forth.
This procedure could require hundreds of steps to finish. Imagine a long chain snaking through the opponent’s territory on a board with 200 moves played already. To speed this up, you can explicitly track all directly connected stones as a unit.

##### 3.1.2. Tracking connected groups of stones in Go: strings
You saw in the preceding section that viewing stones in isolation can lead to an increased computational complexity. Instead, you’ll keep track of groups of connected stones of the same color `and their liberties` at the same time. Doing so is much more efficient when implementing game logic.

You call a group of connected stones of the same color a `string of stones`, or simply a string, as shown in figure 3.1. You can build this structure efficiently with the Python `set` type as in the following `GoString` implementation, which you also put into goboard_slow.py.

In [291]:
# Listing 3.4. Encoding strings of stones with `set`
class GoString():
    """
    >>> liberties=[8]
    >>> g=GoString('w','abc',liberties)
    >>> g.remove_liberty(8)
    >>> g.num_liberties
    0
    >>> g.remove_liberty(8)
    Traceback (most recent call last):
        ...
    KeyError: 8
    
    Furthermore, note the merged_with method of GoString, which is called when a player connects two of its groups by placing a stone.
    >>> len(g.merged_with(g).liberties)
    0
    >>> g.add_liberty(9)
    >>> g.liberties
    {9}
    """
    def __init__(self, color, stones, liberties):
        self.color = color
        self.stones = set(stones)
        self.liberties = set(liberties)

    def remove_liberty(self, point):
        self.liberties.remove(point)

    def add_liberty(self, point):
        self.liberties.add(point)

    def merged_with(self, go_string):
        assert go_string.color == self.color
        combined_stones = self.stones | go_string.stones
        return GoString(
            self.color,
            combined_stones,
            (self.liberties | go_string.liberties) - combined_stones)

    @property
    def num_liberties(self):
        return len(self.liberties)

    def __eq__(self, other):
        return isinstance(other, GoString) and \
            self.color == other.color and \
            self.stones == other.stones and \
            self.liberties == other.liberties



__Figure 3.1. In this Go game, black has three strings of stones, and white has two. The large white string has six liberties, and the single white stone has only three.__
 <p align="center">
     <img src="https://drek4537l1klr.cloudfront.net/pumperla/Figures/03fig01.jpg" alt="string">
</p>

Note that `GoString` directly tracks its own liberties, and you can access the number of liberties at any point by calling `num_liberties`, which is much more efficient than the preceding naive approach starting from individual stones.

Also, you have the ability to add and remove liberties from the given string by using `remove_liberty` and `add_liberty`. Liberties of a string will usually decrease when opponents play next to this string, and increase when this or another group captures opposing stones adjacent to this string.

Furthermore, note the `merged_with` method of `GoString`, which is called when a player connects two of its groups by placing a stone.

##### 3.1.3. Placing and capturing stones on a Go board
After having discussed stones and strings of stones, the natural next step is to discuss how to place stones on the board. Using the `GoString` class from listing 3.4, the algorithm for placing a stone looks like this:

1. Merge any adjacent strings of the same color.
2. Reduce liberties of any adjacent strings of the opposite color.
3. If any opposite-color strings now have zero liberties, remove them.

Also, if the newly created string has zero liberties, you reject the move. This leads naturally to the following implementation of the Go `Board` class, which you also place at goboard_slow.py. You allow boards to have any number of rows or columns by instantiating them with `num_rows` and `num_cols` appropriately. To keep track of the board state internally, you use the private variable `_grid`, a dictionary you use to store strings of stones. First off, let’s initiate a Go board instance by specifying its size.



In [292]:
# Listing 3.5. Creating a Go Board instance
class Board():
    """ 
    You allow boards to have any number of rows or columns by instantiating them with num_rows and num_cols appropriately
    >>> b=Board()
    >>> b.num_rows
    19
    >>> b._grid
    {}
    >>> isinstance(b._grid,dict)
    True
     """
    def __init__(self, num_rows=19, num_cols=19):
        self.num_rows = num_rows
        self.num_cols = num_cols
        self._grid = {}



Next, we discuss the `Board` method to place stones. In `place_stone`, you first have to inspect all neighboring stones of a given point for liberties.

In [293]:
# Listing 3.6. Checking neighboring points for liberties
def place_stone(self, player, point):
    """ 
    >>> from dlgo.gotypes import Point
    >>> from dlgo.goboard_slow import Board
    >>> b=Board()
    >>> p=Point(1,col=2)
    >>> b.place_stone('w',p)
    >>> b._grid[p].num_liberties
    3
    """
    assert self.is_on_grid(point)
    assert self._grid.get(point) is None
    adjacent_same_color = []
    adjacent_opposite_color = []
    liberties = []
    for neighbor in point.neighbors():
        if not self.is_on_grid(neighbor):
            continue
        neighbor_string = self._grid.get(neighbor)
        if neighbor_string is None:
            liberties.append(neighbor)
        elif neighbor_string.color == player:
            if neighbor_string not in adjacent_same_color:
                adjacent_same_color.append(neighbor_string)
        else:
            if neighbor_string not in adjacent_opposite_color:
                adjacent_opposite_color.append(neighbor_string)
    new_string = GoString(player, [point], liberties)



Note that the first two lines in listing 3.6 use utility methods to check whether the point is within bounds for the given board and that the point hasn’t been played yet. These two methods are defined as follows.



In [294]:
# Listing 3.7. Utility methods for placing and removing stones
def is_on_grid(self, point):
    """ 
    >>> from dlgo.gotypes import Point
    >>> from dlgo.goboard_slow import Board
    >>> b=Board()
    >>> p=Point(row=1,col=2)
    >>> b.is_on_grid(p)
    True
    >>> b.get_go_string(p)==b._grid.get(p)
    True
    >>> b._grid
    {}
     """
    return 1 <= point.row <= self.num_rows and \
        1 <= point.col <= self.num_cols

def get(self, point):
    string = self._grid.get(point)
    if string is None:
        return None
    return string.color

def get_go_string(self, point):
    string = self._grid.get(point)
    if string is None:
        return None
    return string



Note that you also define `get_go_string` to return the string of stones associated with a given point. This functionality can be helpful in general, but it’s particularly valuable to prevent `self-capture`, which we’ll discuss in more detail in section 3.2.

Continuing with the definition of `place_stone` in listing 3.6, right after defining `new_string`, you follow the outlined three-step approach shown next.

Listing 3.8. Continuing our definition of place_stone

```python
for same_color_string in adjacent_same_color:
    new_string = new_string.merged_with(same_color_string)
for new_string_point in new_string.stones:
    self._grid[new_string_point] = new_string
for other_color_string in adjacent_opposite_color:
    other_color_string.remove_liberty(point)
for other_color_string in adjacent_opposite_color:
    if other_color_string.num_liberties == 0:
        self._remove_string(other_color_string)
```

Now, the only thing missing in our definition of a Go board is how to remove a string of stones, as required in `remove_string` in the last line of listing 3.8. This is fairly simple, as shown in listing 3.9, but you have to keep in mind that other stones might gain liberties when removing an enemy string. For instance, in figure 3.2, you can see that black can capture a white stone, thereby gaining one additional liberty for each black string of stones.

In [ ]:
# Listing 3.9. Continuing our definition of place_stone
def _remove_string(self, string):
    for point in string.stones:
        for neighbor in point.neighbors():
            neighbor_string = self._grid.get(neighbor)
            if neighbor_string is None:
                continue
            if neighbor_string is not string:
                neighbor_string.add_liberty(point)
        self._grid[point] = None



__Figure 3.2. Black can capture a white stone, thereby regaining a liberty for each of the Go strings adjacent to the captured stone.__

![gain liberties by capture](https://drek4537l1klr.cloudfront.net/pumperla/Figures/03fig02_alt.jpg)


This definition concludes our `Board` implementation.



In [ ]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=38)